# 2.2 函数

日期： 2021.10.17

作者：陈久宁

大纲：

- 函数的定义方式
- 高阶函数
- `do` 语法

如果你有其他语言的基础的话，不妨先阅读文档 [Julia 与其他语言的差异](https://docs.julialang.org/en/v1/manual/noteworthy-differences/)

### 函数的标准定义

In [8]:
function addmul(x, y, z)
    tmp = x + y
    return tmp * z
end

addmul (generic function with 1 method)

In [9]:
addmul(1, 2, 3)

9

不写 `return` 时，默认返回最后一行的结果

In [10]:
function addmul(x, y, z)
    tmp = x + y
    tmp * z
end

addmul (generic function with 1 method)

In [11]:
addmul(1, 2, 3)

9

### 单行函数 one liner

Julia 下有很大部分的函数都是单行函数

In [31]:
addmul(x, y, z) = (x + y) * z

addmul (generic function with 1 method)

In [32]:
addmul(1, 2, 3)

9

### 位置参数默认值

位置参数的默认值通过 `=` 给出

In [37]:
function clamp(x, lo, hi) # [lo, hi]
    # 将输入 x 裁剪到 [lo, hi] 范围内
    if x < lo
        return lo
    elseif x > hi
        return hi
    else
        return x
    end
end

clamp (generic function with 3 methods)

In [38]:
@show clamp(0.3, 0, 1)
@show clamp(-0.3, 0, 1)
@show clamp(1.3, 0, 1)

clamp(0.3, 0, 1) = 0.3
clamp(-0.3, 0, 1) = 0
clamp(1.3, 0, 1) = 1


1

三元表达式
```julia
cond ? true_rst : false_rst
```

等价于

```julia
if cond
    true_rst
else
    false_rst
end
```

In [39]:
clamp(x, lo=0, hi=1) = x < lo ? lo : x > hi ? hi : x

clamp (generic function with 3 methods)

In [40]:
@show clamp(0.3)
@show clamp(-0.3)
@show clamp(1.3)

clamp(0.3) = 0.3
clamp(-0.3) = 0
clamp(1.3) = 1


1

广播:

Rule 1: 任意函数和运算符加上 `.` 表示逐元素的运算

In [45]:
clamp.(-0.5:0.2:1.5) # what happens if you do `clamp(-0.5:0.1:1.5)`?

11-element Vector{Real}:
 0
 0
 0
 0.1
 0.3
 0.5
 0.7
 0.9
 1
 1
 1

对于这种简单的场景来说，广播也可以等价地用列表表达式 (list comprehension) 来写

In [46]:
[clamp(x) for x in -0.5:0.2:1.5] # list comprehension

11-element Vector{Real}:
 0
 0
 0
 0.1
 0.3
 0.5
 0.7
 0.9
 1
 1
 1

如果用 `()` 的话，拿到的就是一个迭代器：具体的数值会在真正使用的时候才计算出来

In [47]:
(clamp(x) for x in -0.5:0.2:1.5)

Base.Generator{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}}, typeof(clamp)}(clamp, -0.5:0.2:1.5)

与 `=>` 结合，构造字典等类型的时候会非常方便

In [48]:
Dict((x => clamp(x) for x in -0.5:0.2:1.5))

Dict{Float64, Real} with 11 entries:
  0.3  => 0.3
  -0.3 => 0
  0.7  => 0.7
  1.1  => 1
  -0.1 => 0
  0.9  => 0.9
  -0.5 => 0
  1.3  => 1
  0.5  => 0.5
  1.5  => 1
  0.1  => 0.1

In [50]:
f_list() = Dict([x => clamp(x) for x in -0.5:0.2:1.5])
f_generator() = Dict((x => clamp(x) for x in -0.5:0.2:1.5))

# 利用 Generator 的话，并不需要创建一个中间变量来存储所有的结果，
# 因此在内存使用上是比较高效的
@btime f_list()
@btime f_generator()
nothing

  3.836 μs (51 allocations: 4.38 KiB)
  1.462 μs (25 allocations: 3.41 KiB)


与 Python 的差别：Julia 函数的默认值发生在函数调用的时候

In [161]:
function push_to_list(x, l = [])
    @show l
    push!(l, x)
end

push_to_list (generic function with 2 methods)

默认值的创建发生在函数调用的时候意味着多次调用拿到的是相同的结果：

In [163]:
push_to_list(1)
push_to_list(1);

l = Any[]
l = Any[]


注：纯函数：`X==Y` 能够推出 `f(X) == f(Y)` 成立。函数的默认值发生在函数调用的时候能够实现纯函数式的代码。

### 可变参数

In [51]:
mysum(t::Tuple) = mysum(t...)
function mysum(x, y...)
    z = copy(x)
    for i in y
        z += i
    end
    return z
end

mysum (generic function with 2 methods)

In [52]:
mysum((1, 2, 3))

6

In [53]:
mysum(1, 2, 3)

6

In [54]:
@show mysum(1)
@show mysum(1, 2)
@show mysum(1, 2, 3)

mysum(1) = 1
mysum(1, 2) = 3
mysum(1, 2, 3) = 6


6

### 关键词参数

以 `;` 分隔位置参数和关键词参数

In [55]:
clamp(x; lo::Real=0, hi::Real=1) = clamp(x, lo, hi)

clamp (generic function with 3 methods)

In [56]:
clamp.(0.0:0.1:0.7, lo=1.0, hi=0.5)

8-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

注：Julia 将位置参数和关键字参数划分的非常清楚，背后的一个原因是因为 Julia 的关键字参数不参与多重派发

In [57]:
mul(x, y) = x * y
mul(x; z) = x * z

mul (generic function with 2 methods)

In [61]:
mul(2, 3)

6

In [63]:
mul(2; z=3)

6

In [66]:
mul(2, 3; z=4) # mul(x, y) 方法不存在关键字参数

LoadError: MethodError: no method matching mul(::Int64, ::Int64; z=4)
[0mClosest candidates are:
[0m  mul(::Any, ::Any) at In[57]:1[91m got unsupported keyword argument "z"[39m
[0m  mul(::Any; z) at In[57]:2

覆盖 `mul(x)` 方法的话，就不能再使用 `z` 关键字了

In [68]:
mul(x; y) = x * y

mul (generic function with 2 methods)

In [69]:
mul(2; y=3)

6

In [71]:
mul(2; z=3) # 原来的 mul(x; z) 方法被新的 mul(x; y) 方法覆盖了

LoadError: UndefKeywordError: keyword argument y not assigned

### 运算符也是函数

In [72]:
+(1, 2) # 1 + 2

3

### 参数传递时是传引用

In [73]:
function change_first!(x)
    x[1] = 0
    return x
end

x = [1, 2, 3]
change_first!(x)
x # [1, 2, 3] or [0, 2, 3]?

3-element Vector{Int64}:
 0
 2
 3

关于函数名后面的 `!`: 这是一个[命名上的约定](https://docs.julialang.org/en/v1/manual/variables/#Stylistic-Conventions) 用来告诉其他程序员：这个函数可能会修改输入的类型。

比如说：`sort!`/`sort`

### 高阶函数 `map`, `reduce`, `mapreduce`

`map(f, X, Y, ...)` 在一定程度上上等价于

```julia
function trivial_map(f, X, Y)
    n = min(length(X), length(Y))
    out = zeros(n)
    for i in 1:n
        out[i] = f(X[i], Y[i])
    end
    return out
end
```

In [75]:
map(clamp, -0.3:0.2:1.3)

9-element Vector{Real}:
 0
 0
 0.1
 0.3
 0.5
 0.7
 0.9
 1
 1

In [76]:
map(+, 1:5, 2:6)

5-element Vector{Int64}:
  3
  5
  7
  9
 11

`reduce` 在一定程度上等价于

```julia
function trivial_reduce(f, X)
    out = zero(eltype(X))
    for i in 1:length(X)
        out = f(out, X[i])
    end
    return out
end
```

In [72]:
reduce(+, [1, 2, 3]) # sum([1, 2, 3])

6

In [73]:
mapreduce(abs2, +, [1, 2, 3]) # sum(abs2.([1, 2, 3]))

14

Julia 下还存在很多这种将函数作为输入的函数

In [78]:
sum(abs2, [1, 2, 3])

14

`foreach(f, X)` 等价于

```julia
for x in X
    f(x)
end
```

In [82]:
foreach(println, [1, 2, 3])

1
2
3


`filter(f, X)` 等价于

```julia
rst = []
for x in X
    if f(x)
        push!(rst, x)
    end
end
```

In [83]:
filter(isodd, 1:9)

5-element Vector{Int64}:
 1
 3
 5
 7
 9

### 匿名函数

Julia 下各种函数都可以作为另一个函数的输入，因此匿名函数变得十分有用

`x -> f(x)` 等价于 `g(x) = f(x)`

唯一的差别在于函数名是不确定的（由系统生成）

In [85]:
f2(x) = x^2 # 名字为 f2

f2 (generic function with 1 method)

In [86]:
f = x-> x^2 # 名字 #?? 由系统生成, f 只是变量名 （并非是函数名）

#36 (generic function with 1 method)

In [89]:
map(x -> x>0.5 ? 1.0 : 0.0, 0.0:0.2:1.0)

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0

### `do`-block

有时候匿名函数写的太长了也不好，所以 Julia 提供了 `do` 语法

```julia
f(X) do i
   g(i)
end
```

等价于

```julia
f(i->g(i), X)
```

In [92]:
# 以下两者等价
map(x -> x > 0.5 ? 1.0 : 0.0, 0.0:0.2:1.0)
map(0.0:0.2:1.0) do x
    if x > 0.5
        return 1.0
    else
        return 0.0
    end
end

6-element Vector{Float64}:
 0.0
 0.0
 0.0
 1.0
 1.0
 1.0

最佳实践：关于 `do` 的一个典型用法是用来处理文件读写，保证文件在处理完之后被正常关闭。

```julia
fio = open(filename, "w")
do_something(fio)
close(fio
```

以上这种写法并不能够保证文件一定被正常关闭：比如`do_something` 执行到一半报错了。

See also [RAII(Resource acquisition is initialization)](https://en.wikipedia.org/wiki/Resource_acquisition_is_initialization)

`python` 为此引入了一个特殊的语法格式：

```python
with open("out.txt", "w") as fio:
    print(fio, "hello")
    error("...")
```

而在 Julia 下通过 `do` 就可以实现

In [96]:
# do 里面的内容执行完之后，文件会自动被关闭
open("out.txt", "w") do io
    println(io, "hello")
    println(io, "world")
end

shell mode `;`

In [88]:
;cat out.txt

hello
world


这背后实际上是也是构造了一个高阶函数：

In [99]:
function myopen(f, filename, mode)
    local fio
    try
        fio = open(filename, mode)
        f(fio)
    finally
        close(fio)
    end
end

myopen (generic function with 1 method)

In [101]:
myopen("out.txt", "w") do io
    println(io, "world")
    println(io, "hello")
end

In [94]:
;cat out.txt

world
hello


换句话说，并不需要像 Python 一样的 `with` 来处理，而只需要符合 `do` 语法就可以了.

`do` 后面加哪些变量以及每个变量的意义，取决于高阶函数的具体设计

In [102]:
map(+, 1:5, 2:6)

5-element Vector{Int64}:
  3
  5
  7
  9
 11

In [103]:
map(1:5, 2:6) do x, y
    x + y
end

5-element Vector{Int64}:
  3
  5
  7
  9
 11

### Julia 没有类方法，也不存在面向对象编程的代码风格

类方法本质上可以理解成单重派发，而 Julia 的多重派发是一种更一般化的编程模式。

In [145]:
struct Point{T}
    x::T
    y::T
end
struct Point3D{T}
    x::T
    y::T
    z::T
end

In [146]:
p2, q2 = Point(3, 4), Point(0, 0)
p3, q3 = Point3D(1, 2, 1), Point3D(1, 5, 5)

(Point3D{Int64}(1, 2, 1), Point3D{Int64}(1, 5, 5))

如果需要实现一个函数 `dist`，来计算所有不同可能组合：

- `dist(p2, q2)`
- `dist(p2, p3)`
- `dist(p3, p2)`
- `dist(p3, q3)`

在 Python 下会如何实现？在 Julia 下会如何实现？

Julia 实现方式 1: 针对每一种组合写一个方法

In [147]:
dist(p::Point, q::Point) = sqrt((p.x-q.x)^2+(p.y-q.y)^2)
dist(p::Point, q::Point3D) = sqrt((p.x-q.x)^2+(p.y-q.y)^2 + q.z^2)
dist(p::Point3D, q::Point) = sqrt((p.x-q.x)^2+(p.y-q.y)^2 + p.z^2)
dist(p::Point3D, q::Point3D) = sqrt((p.x-q.x)^2+(p.y-q.y)^2 + (p.z-q.z)^2)

dist (generic function with 4 methods)

In [148]:
@show dist(p2, q2)
@show dist(p2, p3)
@show dist(p3, p2)
@show dist(p3, q3)

dist(p2, q2) = 5.0
dist(p2, p3) = 3.0
dist(p3, p2) = 3.0
dist(p3, q3) = 5.0


5.0

假如出现了第三个类...

In [149]:
struct Point1D{T}
    x::T
end

In [150]:
p1, q1 = Point1D(1), Point1D(3)

(Point1D{Int64}(1), Point1D{Int64}(3))

难道要写 M^N (M为函数参数类型的可能性，N为函数参数的数目）个方法？(在这里是 3^2==9 个)

Julia 实现方式 2: 拆分成两个正交的阶段

In [151]:
promote_point(p::Point1D, q::Point) = Point(p.x, 0), q
promote_point(p::Point1D, q::Point3D) = Point3D(p.x, 0, 0), q
promote_point(p::Point, q::Point3D) = Point3D(p.x, 0, 0), q

promote_point(p::T, q::T) where T = p, q
promote_point(p, q) = promote_point(q, p)

promote_point (generic function with 5 methods)

In [152]:
dist2(p::Point1D, q::Point1D) = p.x - q.x
dist2(p::Point, q::Point) = sqrt((p.x-q.x)^2+(p.y-q.y)^2)
dist2(p::Point3D, q::Point3D) = sqrt((p.x-q.x)^2+(p.y-q.y)^2+(p.z-q.z)^2)
dist2(p, q) = dist2(promote_point(p, q)...)

dist2 (generic function with 4 methods)

因此只需要 4 个 `dist` 方法就可以定义完整的功能，并且实际上代码结构也会更加简洁。

这里 `promote_point` 没有算进去，是可以它被其他相似的方法复用，比如说某一天需要再实现一个支持各种类型的 `point_add`，就不需要再写一遍了。

基于这个原因，Julia 中随处可以见到这种代码风格，并且也作为最佳实践记录在文档中 [将代码设计正交化](https://docs.julialang.org/en/v1/manual/methods/#man-methods-orthogonalize)

In [153]:
@show dist2(p1, q1)
@show dist2(p1, p2)
@show dist2(p1, p3)

@show dist2(p2, p1)
@show dist2(p2, q2)
@show dist2(p2, p3)

@show dist2(p3, p1)
@show dist2(p3, p2)
@show dist2(p3, q3)

dist2(p1, q1) = -2
dist2(p1, p2) = 4.47213595499958
dist2(p1, p3) = 2.23606797749979
dist2(p2, p1) = 4.47213595499958
dist2(p2, q2) = 5.0
dist2(p2, p3) = 3.0
dist2(p3, p1) = 2.23606797749979
dist2(p3, p2) = 3.0
dist2(p3, q3) = 5.0


5.0